# Post-QUICHE processing

In [ ]:
import os
import pandas as pd
import numpy as np
import quiche as qu
import scanpy as sc
import anndata
import dill

In [ ]:
base_dir = "../data"
cell_tab_path = os.path.join(base_dir, "tables", "cell_table_size_normalized.csv")
metadata_path = os.path.join(base_dir, "tables", "metadata.csv")

cell_tab = pd.read_csv(cell_tab_path)
metadata = pd.read_csv(metadata_path)
data = pd.merge(cell_tab, metadata, on="fov", how="left")

In [ ]:
# get quiche output
quiche_out_path = os.path.join(base_dir, "spatial_analysis", "quiche", "quiche_hivpos_hivneg_sketch50000.pkl")
with open(quiche_out_path, 'rb') as file:
    quiche_op = dill.load(file)

## Inspect QUICHE object

In [ ]:
mdata = quiche_op.mdata

In [ ]:
niche_scores = qu.tl.filter_niches(quiche_op,
                                   thresholds = {'logFC': {'median' : [-1, 1]}, 'SpatialFDR': {'median' : 0.05}},
                                   min_niche_count = 5,
                                   annotation_key = 'quiche_niche_neighborhood')

niche_metadata = qu.tl.compute_niche_metadata(quiche_op,
                                            niches = list(niche_scores.index),
                                            annotation_key = 'quiche_niche_neighborhood',
                                            patient_key = 'sample_id',
                                            condition_key = 'status',
                                            niche_threshold = 0,
                                            condition_type  = 'binary',
                                            metrics = ['logFC', 'SpatialFDR', 'PValue'])

In [ ]:
logfc_key = 'logFC'
pvalue_key = 'SpatialFDR'
alpha = 0.05

mdata = quiche_op.mdata
nhood_adata = mdata['quiche'].T.copy()

niches = niche_scores.index
annotation_key = 'quiche_niche_neighborhood'
niches = list(set(niches).intersection(set(niche_metadata[annotation_key].unique()))) #ensures niches are in metadata and pass niche_threshold
nhood_adata = nhood_adata[np.isin(nhood_adata.obs[annotation_key], niches)]

sorted_annos = (nhood_adata.obs[[annotation_key, logfc_key]].groupby(annotation_key).mean().sort_values(logfc_key, ascending=True).index)

anno_df = nhood_adata.obs[[annotation_key, logfc_key, pvalue_key]].copy()
anno_df["is_signif"] = anno_df[pvalue_key] < alpha
anno_df = anno_df[anno_df[annotation_key] != "nan"]

## Save data for beeswarm

In [ ]:
mdata = quiche_op.mdata
nhood_adata = mdata['quiche'].T.copy()

annotation_key = 'quiche_niche_neighborhood'
logfc_key = 'logFC'
sorted_annos = (nhood_adata.obs[[annotation_key, logfc_key]].groupby(annotation_key).mean().sort_values(logfc_key, ascending=True).index)

## Save quiches as cell table

In [ ]:
mdata = quiche_op.mdata
cell_tab = mdata['quiche'].var
save_tab = cell_tab[['fov','label','cell_meta_cluster','quiche_niche_neighborhood','logFC','SpatialFDR']]
save_tab.to_csv(os.path.join(base_dir, "spatial_analysis", "quiche", "quiche_cell_tab.csv"), index=False)

## Save significiant quiches as table

In [ ]:
niche_scores_med = qu.tl.filter_niches(quiche_op,
                                   thresholds = {'logFC': {'median' : [-1, 1]}, 'SpatialFDR': {'median' : 0.05}},
                                   min_niche_count = 5,
                                   annotation_key = 'quiche_niche_neighborhood')
niche_scores_med.to_csv(os.path.join(base_dir, "spatial_analysis", "quiche", "quiche_sig_niches_median.csv"), index=True)

niche_scores_mean = qu.tl.filter_niches(quiche_op,
                                   thresholds = {'logFC': {'mean' : [-1, 1]}, 'SpatialFDR': {'mean' : 0.05}},
                                   min_niche_count = 5,
                                   annotation_key = 'quiche_niche_neighborhood')
niche_scores_mean.to_csv(os.path.join(base_dir, "spatial_analysis", "quiche", "quiche_sig_niches_mean.csv"), index=True)

## Make network graph

In [ ]:
colors_dict = {'myeloid':'#4DCCBD', 'lymphoid':'#279AF1', 'other_immune':'#B07AA1', 'other':'#FF8484', 'stromal':'#F9DC5C'}

lineage_dict = {'APC':'myeloid', 'B':'lymphoid', 'CD11c': 'myeloid', 'CD11c_CD14': 'myeloid', 'CD11c_CD68': 'myeloid', 'CD14': 'myeloid',
                'CD14_CD68_CD163': 'myeloid', 'CD3_other': 'lymphoid', 'CD4T': 'lymphoid', 'CD68': 'myeloid', 'CD8T': 'lymphoid',
                'Endothelial':'stromal', 'FDC': 'stromal', 'Foxp3': 'lymphoid', 'Immune_other':'other_immune', 'Mast':'other_immune',
                'NK':'lymphoid', 'Neutrophils':'other_immune', 'Other':'other', 'SMA':'stromal', 'Tfh':'lymphoid', 'Unassigned':'other'}

cell_ordering = ['APC', 'CD11c', 'CD11c_CD14', 'CD11c_CD68', 'CD14',
                 'CD14_CD68_CD163', 'CD68', 'B', 'CD8T', 'CD4T', 'Tfh',
                 'Foxp3', 'CD3_other', 'NK', 'Mast', 'Neutrophils',
                 'Immune_other', 'FDC', 'SMA', 'Endothelial', 'Other', 'Unassigned']

##niche network for patients that did not relapse, ie logFC < 0
niche_metadata_neg = niche_metadata[(niche_metadata['mean_logFC'] < 0) & (niche_metadata['n_patients_niche'] > 1)]
G1 = qu.tl.compute_niche_network(niche_df = niche_metadata_neg,
                          colors_dict = colors_dict,
                          lineage_dict = lineage_dict,
                          annotation_key = 'quiche_niche_neighborhood')

qu.pl.plot_niche_network_donut(G = G1,
                               figsize=(6, 6),
                               node_order = cell_ordering,
                               centrality_measure = 'eigenvector',
                               colors_dict = colors_dict,
                               lineage_dict=lineage_dict, 
                               donut_radius_inner = 1.15,
                               donut_radius_outer = 1.25,
                               edge_cmap = 'bone_r',
                               edge_label = 'Samples',
                               save_directory = '../data/spatial_analysis/quiche/plots',
                               filename_save = 'hivneg_donut')

##niche network for patients that relapsed, ie logFC > 0
niche_metadata_pos = niche_metadata[(niche_metadata['mean_logFC'] > 0) & (niche_metadata['n_patients_niche'] > 1)]
G2 = qu.tl.compute_niche_network(niche_df = niche_metadata_pos,
                          colors_dict = colors_dict,
                          lineage_dict = lineage_dict,
                          annotation_key = 'quiche_niche_neighborhood')

qu.pl.plot_niche_network_donut(G = G2,
                               figsize=(6, 6),
                               node_order = cell_ordering,
                               centrality_measure = 'eigenvector',
                               colors_dict = colors_dict,
                               lineage_dict=lineage_dict, 
                               donut_radius_inner = 1.15,
                               donut_radius_outer = 1.25,
                               edge_cmap = 'bone_r',
                               edge_label = 'Samples',
                               save_directory = '../data/spatial_analysis/quiche/plots',
                               filename_save = 'hivpos_donut')